In [2]:
max1demand=11   #use 1 more than actual demand
max2demand=11    #use 1 more than actual demand
sparti=[]  #start with null
spartj=[]  #start with null
spartk=[]  #start with null
spartl=[]  #start with null
spartm=[]
spartn=[]
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:4          #Create an index to call
    for j in 1:4
        for k in 1:max1demand
            for l in 1:max2demand
                for m in 1:2
                    for n in 1:2
          #statei(i+1) = s
            sparti = push!(sparti,i)  #for i in ijkl add to sparti definition
            spartj = push!(spartj,j) #for j in ijkl add to spartj definition
            spartk = push!(spartk,k) #for k in ijkl add to spartk definition
            spartl = push!(spartl,l) #for l in ijkl add to spartl definition
            spartm = push!(spartm,m) #for m in ijkl add to spartl definition
            spartn = push!(spartn,n) #for n in ijkl add to spartl definition            
            sidx= push!(sidx,s)      #create index in sidx
            s=s+1
                    end 
                end
            end
        end
    end
end

#sparti
#spartj
#spartk
#[sparti,spartj,spartk]
#sidx

#idxactive=  (sparti.>=4) .& (spartj+spartk+spartl.==3) ##(Wrapped into regional indices)
idxidle= (sparti.<3) .& (spartj.<3) .& (spartk+spartl .>2)   #index all ijkl where idling cannot occur
idxidle2= (sparti.>=3) .& (spartj.<3) .& (spartk+spartl .>3) #index where more than 2 calls exist but ambulance 2 is not busy
idxidle1= (spartj.>=3) .& (sparti.<3) .& (spartk+spartl .>3) #index where more than 2 calls exist but ambulance 1 is not busy
idxregion1= ((sparti.==3) .| (spartj.==3)) .& (spartk.==1)      #index all ijkl where region 1 is active but there are no calls in region 1 to service
idxregion2= ((sparti.==4) .| (spartj.==4)) .& (spartl.==1)      #index all ijkl where region 2 is active but there are no calls in region 2 to service
idxregion11= ((sparti.==3) .& (spartj.==3)) .& (spartk.==2)      #index all ijkl where region 1 is active with both ambulances but there is only 1 call in region 1 to service
idxregion21= ((sparti.==4) .& (spartj.==4)) .& (spartl.==2)      #index all ijkl where region 2 is active with both ambulances but there is only 1 call in region 2 to service


idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxidle1  .| idxidle2 .| idxregion11 .| idxregion21 #index of the previous 3 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartm, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartn, BitArray(idxDEL)) #delete idxDEL  
sidx= [1:length(sparti);]       #number of states

1640-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
   11
   12
   13
    ⋮
 1629
 1630
 1631
 1632
 1633
 1634
 1635
 1636
 1637
 1638
 1639
 1640

In [3]:
μMat= zeros(length(sidx),1)

1640×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [4]:
#Add Arrival Code
λst=.5  #Define overall Arrival Rate for every zone
#λ1=.5  #Define Arrival Rate for Zone1
#λ2=.5  #Define Arrival Rate for Zone2                
μMatr=[1.5 1; 
        1 1.5]
# μ11=1  #Define Service Rate from Zone1 to Zone1
# μ12=.7 #Define Service Rate from Zone1 to Zone2
# μ21=.7 #Define Service Rate from Zone2 to Zone1
# μ22=1  #Define Service Rate from Zone2 to Zone2
matrix= zeros(length(sidx),length(sidx))
μ1idx=zeros(length(sidx))
μ2idx=zeros(length(sidx))
λ1idx=zeros(length(sidx))
λ2idx=zeros(length(sidx))
using LinearAlgebra


In [11]:
N=["λ" "μ1" "μ2" "Throughput" "Deliveries/Hour" "Avg. # in Sys" "Variance" "Avg. Time in Sys" "% of Time 0 calls" "% of Time 1 call" "% of Time 2 Call" "Compliance Rate"]
μ1=5.5
λst=0 #Define overall Arrival Rate for every zone


for i in 1:1
     λst= .5
     μ1=2.75
  for j in 1:8
      μ1=μ1-.25

μ2=.9*μ1
μMatr=[μ1 μ2; 
       μ2 μ1]
matrix= zeros(length(sidx),length(sidx))
λ1idx=zeros(length(sidx))
λ2idx=zeros(length(sidx)) 
μMat= zeros(length(sidx),1)

for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1=0
    nextRegionFrom2=0
    nexts=0
    λ= 0    
 #   Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2        
        if Zoneflag1 == 1 
            if Zoneflag2>1
             nextZoneflag1=3
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=1
             nextRegionFrom2=RegionFrom2
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ       
             λ1idx[s]=1 
            else 
             λ=λst/2
             nextRegionFrom1=1
             nextRegionFrom2=1
             idx= (sparti.==3) .& (spartj.==1) .& (spartk.==nextCall1) .&(spartl.==nextCall2).& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==1) .& (spartj.==3) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
             λ1idx[s]=1 
            end
        elseif Zoneflag1 == 2 
            if Zoneflag2>2
             nextZoneflag1=3
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=2
             nextRegionFrom2=RegionFrom2
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
             λ1idx[s]=1    
            elseif Zoneflag2==1
             nextZoneflag1=Zoneflag1
             nextZoneflag2=3
             nextRegionFrom1=RegionFrom1
             nextRegionFrom2=1 
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
             λ1idx[s]=1 
            else 
             λ=λst/2   
             nextRegionFrom1=2
             nextRegionFrom2=2                 
             idx= (sparti.==3) .& (spartj.==2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==2) .& (spartj.==3) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             λ1idx[s]=1
            end            
        elseif Zoneflag1>2 && Zoneflag2<=2
            nextZoneflag1 = Zoneflag1  
            nextZoneflag2 = 3 
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=Zoneflag2
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ        
            λ1idx[s]=1 
        else Zoneflag1>2 && Zoneflag2>2
            nextZoneflag1 = Zoneflag1
            nextZoneflag2 = Zoneflag2
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=RegionFrom2
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ  
            λ1idx[s]=1 
        end        
    end
end
         for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]    
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1=0
    nextRegionFrom2=0    
    nexts=0
    λ= 0  
  #   Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1               
        if Zoneflag1 == 2 
            if Zoneflag2!=2
             nextZoneflag1=4
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=2
             nextRegionFrom2=RegionFrom2
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
                λ2idx[s]=1 
            else 
             λ=λst/2   
             nextRegionFrom1=2
             nextRegionFrom2=2            
             idx= (sparti.==4) .& (spartj.==2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==2) .& (spartj.==4) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
                λ2idx[s]=1
            end
        elseif Zoneflag1 == 1
            if Zoneflag2>2
             nextZoneflag1=4
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=1
             nextRegionFrom2=RegionFrom2                            
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
                λ2idx[s]=1 
            elseif Zoneflag2==2                
             nextZoneflag1=Zoneflag1
             nextZoneflag2=4
             nextRegionFrom1=RegionFrom1
             nextRegionFrom2=2                  
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ    
                λ2idx[s]=1 
            else 
             λ=λst/2   
             nextRegionFrom1=1
             nextRegionFrom2=1                  
             idx= (sparti.==4) .& (spartj.==1) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
             idx= (sparti.==1) .& (spartj.==4) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
            λ2idx[s]=1
            end            
        elseif Zoneflag1>2 && Zoneflag2<=2
            nextZoneflag1 = Zoneflag1  
            nextZoneflag2 = 4
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=Zoneflag2   
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ 
            λ2idx[s]=1 
        
        else Zoneflag1>2 && Zoneflag2>2
            nextZoneflag1 = Zoneflag1
            nextZoneflag2 = Zoneflag2
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=RegionFrom2
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ
            λ2idx[s]=1 
        end
    end
end
        
for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]    
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1= 0
    nextRegionFrom2= 0    
    nexts=0
    RegionService= 0         
     #Service from Region to Region    
     if Zoneflag1 == 3
        if Zoneflag2 == 4
            if Call2 > Call1
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 4
                nextZoneflag2= Zoneflag2
                nextRegionFrom1=1
                nextRegionFrom2=RegionFrom2
                nextCall1= Call1-1
                nextCall2= Call2
            else
                if Call1==2
                    RegionService = μMatr[1,RegionFrom1]
                    nextZoneflag1= 1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1-1
                    nextCall2= Call2
                    nextRegionFrom1=1
                    nextRegionFrom2=RegionFrom2
                else 
                    RegionService = μMatr[1,RegionFrom1]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1-1
                    nextCall2= Call2   
                    nextRegionFrom1=1
                    nextRegionFrom2=RegionFrom2                    
                end 
            end           
        elseif Zoneflag2==3
            if Call2>=Call1-1
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 4
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2
                nextRegionFrom1=1
                nextRegionFrom2=RegionFrom2                  
            elseif Call1>3
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=1
                nextRegionFrom2=RegionFrom2  
            else
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=1
                nextRegionFrom2=RegionFrom2  
            end 
        else 
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=1
                nextRegionFrom2=RegionFrom2
        end      
    elseif Zoneflag1==4  
          if Zoneflag2 == 3
            if Call1 > Call2
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 3
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1
                nextRegionFrom1=2
                nextRegionFrom2=RegionFrom2                 
            else
                if Call2==2
                    RegionService = μMatr[2,RegionFrom1]
                    nextZoneflag1= 2
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1
                    nextCall2= Call2-1   
                    nextRegionFrom1=2
                    nextRegionFrom2=RegionFrom2 
                else 
                    RegionService = μMatr[2,RegionFrom1]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1
                    nextCall2= Call2-1      
                    nextRegionFrom1=2
                    nextRegionFrom2=RegionFrom2 
                end 
            end            
        elseif Zoneflag2==4
            if Call1>=Call2-1
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 3
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1
                nextRegionFrom1=2
                nextRegionFrom2=RegionFrom2 
            elseif Call2>3
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1  
                nextRegionFrom1=2
                nextRegionFrom2=RegionFrom2 
            else 
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 2
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1   
                nextRegionFrom1=2
                nextRegionFrom2=RegionFrom2
            end 
        else 
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 2
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1     
                nextRegionFrom1=2
                nextRegionFrom2=RegionFrom2
        end         
    else
    end        
         idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2).& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
         nexts= sidx[idx] 
         matrix[nexts,s]= matrix[nexts,s] .+ RegionService
         matrix[s,s]= matrix[s,s].- RegionService  
        μMat[s,1]= μMat[s,1] .+ RegionService    
end 
        
        for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]    
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1= 0
    nextRegionFrom2= 0    
    nexts=0
    RegionService= 0         
     #Service from Region to Region    
     if Zoneflag2 == 3
        if Zoneflag1 == 4
            if Call2 > Call1
                RegionService = μMatr[1,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 4
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=1
                nextCall1= Call1-1
                nextCall2= Call2
            else
                if Call1==2
                    RegionService = μMatr[1,RegionFrom2]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= 1
                    nextCall1= Call1-1
                    nextCall2= Call2
                    nextRegionFrom1=RegionFrom1
                    nextRegionFrom2=1
                else 
                    RegionService = μMatr[1,RegionFrom2]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1-1
                    nextCall2= Call2   
                    nextRegionFrom1=RegionFrom1
                    nextRegionFrom2=1                    
                end 
            end           
        elseif Zoneflag1==3
            if Call2>=Call1-1
                RegionService = μMatr[1,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 4
                nextCall1= Call1-1
                nextCall2= Call2
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=1                   
            elseif Call1>3
                RegionService = μMatr[1,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=1  
            else
                RegionService = μMatr[1,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 1
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=1  
            end 
        else 
                RegionService = μMatr[1,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 1
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=1  
        end      
    elseif Zoneflag2==4  
          if Zoneflag1 == 3
            if Call1 > Call2
                RegionService = μMatr[2,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 3
                nextCall1= Call1
                nextCall2= Call2-1
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=2                  
            else
                if Call2==2
                    RegionService = μMatr[2,RegionFrom2]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= 2
                    nextCall1= Call1
                    nextCall2= Call2-1   
                    nextRegionFrom1=RegionFrom1
                    nextRegionFrom2=2 
                else 
                    RegionService = μMatr[2,RegionFrom2]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1
                    nextCall2= Call2-1      
                    nextRegionFrom1=RegionFrom1
                    nextRegionFrom2=2 
                end 
            end            
        elseif Zoneflag1==4
            if Call1>=Call2-1
                RegionService = μMatr[2,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 3
                nextCall1= Call1
                nextCall2= Call2-1
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=2 
            elseif Call2>3
                RegionService = μMatr[2,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1  
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=2 
            else 
                RegionService = μMatr[2,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 2
                nextCall1= Call1
                nextCall2= Call2-1   
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=2 
            end 
        else 
                RegionService = μMatr[2,RegionFrom2]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 2
                nextCall1= Call1
                nextCall2= Call2-1     
                nextRegionFrom1=RegionFrom1
                nextRegionFrom2=2 
        end         
    else
    end        
         idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2).& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
         nexts= sidx[idx] 
         matrix[nexts,s]= matrix[nexts,s] .+ RegionService
         matrix[s,s]= matrix[s,s].- RegionService  
         μMat[s,1]= μMat[s,1] .+ RegionService 
end
matrix[length(sidx),:].= 1; #prep Ahat for matrix inversion
inverse=inv(matrix); #invert Ahat matrix
X=inverse[:,length(sidx)] #extract π from matrix
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
E=spartm[1:length(sidx)]
F=spartn[1:length(sidx)]
M=hcat(sidx,A,B,C,D,E,F,X)     #create readable matrix for state, sparti,spartj,spartk,spartl and pi
Throughput=(λst)*sum(M[λ1idx.==1,8])+(λst)*sum(M[λ2idx.==1,8])
Gain=μMat'*X
L=0
for s in 1:length(sidx)
  L=L+(C[s]+D[s]-2)*X[s]
end
LVar=0
for s in 1:length(sidx)
  LVar=LVar+(C[s]+D[s]-2)^2*X[s]
end
LVar=LVar-L^2
W=L/Throughput 
Midx0= (spartk .+ spartl).==2
ZeroCall=sum(M[Midx0.==1,8])
Midx1= (spartk .+ spartl).==3
OneCall=sum(M[Midx1.==1,8])
Midx2= (spartk .+ spartl).==4
TwoCall=sum(M[Midx2.==1,8])
CR=ZeroCall+OneCall+TwoCall
N=[N; λst μ1 μ2 Throughput Gain L LVar W ZeroCall OneCall TwoCall CR]
        
end
end     

In [12]:
using DataFrames
df= DataFrame(N[2:Int(length(N[:,1])),:], :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.5,2.5,2.25,1.0,1.0,0.429224,0.47024,0.429224,0.659976,0.26937
2,0.5,2.25,2.025,1.0,1.0,0.482848,0.539963,0.482848,0.628722,0.285498
3,0.5,2.0,1.8,1.0,1.0,0.55254,0.635876,0.55254,0.591198,0.302464
4,0.5,1.75,1.575,1.0,1.0,0.647248,0.776692,0.647248,0.545327,0.319397
5,0.5,1.5,1.35,1.0,1.0,0.784505,1.00422,0.784506,0.488009,0.334124
6,0.5,1.25,1.125,0.999999,0.999999,1.00455,1.43243,1.00455,0.414416,0.341275
7,0.5,1.0,0.9,0.999988,0.999988,1.42743,2.49286,1.42745,0.316637,0.326833
8,0.5,0.75,0.675,0.999432,0.999432,2.65535,7.23102,2.65686,0.181097,0.25004


In [13]:
using CSV
CSV.write("C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\GD2AmbStats29MAR_1.csv",df)

"C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\GD2AmbStats29MAR_1.csv"

In [80]:

matrix
sum(matrix)

0.0

In [81]:

sum(matrix)


0.0

In [82]:
matrix[length(sidx),:].= 1; #prep Ahat for matrix inversion

In [83]:
matrix

184×184 Matrix{Float64}:
 -1.0    0.0    0.0    0.0    0.0   0.0  …   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   -1.0    0.0    0.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0   -1.0    0.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0   -1.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0   -1.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0    0.0  -1.0  …   0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.25   0.25   0.25   0.25   0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0    0.0   0.0  …   0.0   0.0   0.0   0.0   0.0  0.0
  0.0    0.0    0.0    0.0    0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
  0.25   0.25   0.25   0.25   0.0   0.0

In [84]:
inverse=inv(matrix); #invert Ahat matrix
#sum(inverse[:,length(sidx)]) #validate that sum of π equals 1

In [85]:
X=inverse[:,length(sidx)] #extract π from matrix

184-element Vector{Float64}:
  0.06474894436756218
  0.0
  0.0
  0.0
 -6.908326698999416e-18
  0.16004915376571952
  0.0
  0.0
  0.013652122375079012
  0.011896288621162592
  0.0
  0.0
  0.0163900303408961
  ⋮
  0.00029768790009148424
  0.00019680718180581
  0.0001968071818058099
  6.249957597225031e-5
  4.43178647396879e-5
  5.326860262439142e-5
  5.326860262439139e-5
  2.245490828544524e-5
  8.550144120779325e-5
  5.0015156886040334e-5
  5.001515688604028e-5
  1.4159080709646998e-5

In [86]:
X=round.(X,digits=10); #round to 10 digits
sum(X)

1.0

In [87]:
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
E=spartm[1:length(sidx)]
F=spartn[1:length(sidx)]
M=hcat(sidx,A,B,C,D,E,F,X)     #create readable matrix for state, sparti,spartj,spartk,spartl and pi

184×8 Matrix{Any}:
   1  1  1  1  1  1  1   0.0647489
   2  1  1  1  1  1  2   0.0
   3  1  1  1  1  2  1   0.0
   4  1  1  1  1  2  2   0.0
   5  1  2  1  1  1  1  -0.0
   6  1  2  1  1  1  2   0.160049
   7  1  2  1  1  2  1   0.0
   8  1  2  1  1  2  2   0.0
   9  1  3  2  1  1  1   0.0136521
  10  1  3  2  1  1  2   0.0118963
  11  1  3  2  1  2  1   0.0
  12  1  3  2  1  2  2   0.0
  13  1  4  1  2  1  1   0.01639
   ⋮              ⋮     
 173  4  4  3  4  1  1   0.000297688
 174  4  4  3  4  1  2   0.000196807
 175  4  4  3  4  2  1   0.000196807
 176  4  4  3  4  2  2   6.24996e-5
 177  4  4  4  3  1  1   4.43179e-5
 178  4  4  4  3  1  2   5.32686e-5
 179  4  4  4  3  2  1   5.32686e-5
 180  4  4  4  3  2  2   2.24549e-5
 181  4  4  4  4  1  1   8.55014e-5
 182  4  4  4  4  1  2   5.00152e-5
 183  4  4  4  4  2  1   5.00152e-5
 184  4  4  4  4  2  2   1.41591e-5

In [66]:
#Percentage of time with zero calls
Midx0= (spartk .+ spartl).==2
sum(M[Midx0.==1,8])

0.44959619640000004

In [67]:
#Percentage of time with 1 call
Midx1= (spartk .+ spartl).==3
sum(M[Midx1.==1,8])

0.33744588920000007

In [101]:
#Percentage of time with 2 calls 
Midx2= (spartj .+ spartk .+ spartl).==5
sum(M[Midx2.==1,6])

0.0310463079

In [102]:
#Percentage of time with 3 calls
Midx3= (spartj .+ spartk .+ spartl).==6
sum(M[Midx3.==1,6])

0.0542819014

In [103]:
#Percentage of time with 4 calls 
Midx4= (spartj .+ spartk .+ spartl).==7
sum(M[Midx4.==1,6])

0.07935388210000001

In [104]:
#Percentage of time with 5 calls 
Midx5= (spartj .+ spartk .+ spartl).==8
sum(M[Midx5.==1,6])

0.1268390758

In [105]:
#Percentage of time with 6 calls 
Midx5= (spartj .+ spartk .+ spartl).==9
sum(M[Midx5.==1,6])

0.17168729919999998

In [106]:
#Percentage of time with 7 calls 
Midx5= (spartj .+ spartk .+ spartl).==10
sum(M[Midx5.==1,6])

0.18873403219999998

In [110]:
#Percentage of time with max calls
MidxMax= (spartj .+ spartk.+ spartl).==(max1demand+max2demand+max3demand)
MaxInSys=sum(M[MidxMax.==1,6])

0.1273207536

In [108]:
#Midxw1Full= ((spartj .+ spartk).<=7).&(spartj.==max1demand)
#Midxw2Full= ((spartj .+ spartk).<=7).&(spartk.==max2demand);


In [282]:
#Define total throughput
#Throughput=((λ1+λ2)*MaxInSys+λ1*sum(M[Midxw2Full.==1,5])+λ2*sum(M[Midxw1Full.==1,5])) ###THIS IS WRONG
Throughput=(μ1Same)*sum(M[μ1idx.==1,6])+(μ1Diff)*sum(M[μ1idx.==2,6])+(μ2Same)*sum(M[μ2idx.==1,6])+(μ2Diff)*sum(M[μ2idx.==2,6]) ## This is somehow wrong- ask Dr. King


1.0544847926

In [88]:
Throughput=(λst)*sum(M[λ1idx.==1,8])+(λst)*sum(M[λ2idx.==1,8])

0.9836298051999999

In [90]:
Gain=μMat'*X

1-element Vector{Float64}:
 0.9836298052